In [1]:
from src.requirements import *

In [ ]:
audio_path = "data"
file_list = dict()

for dir1 in os.listdir(audio_path):
    if os.path.splitext(dir1)[1] == '.tsv':
        continue
    subdir = os.path.join(audio_path, dir1)
    for dir2 in tqdm(os.listdir(subdir)):
        if os.path.splitext(dir2)[1] == '.tsv':
            continue
        path = os.path.join(subdir, dir2)
        waveform, sr = sf.read(path, always_2d=True)
        file_list[path] = sr

In [ ]:
max(file_list.values())

In [2]:
from src.audio_handler import *

In [6]:
train_dataset = AudioDataset(audio_path="data")
train_dl = DataLoader(
    dataset = train_dataset,
    batch_size = 8,
    pin_memory = True,
    # collate_fn = collate_padding, 
    shuffle=True
)

100%|█████████████████████████████████████████████████████████████████████████████| 618/618 [00:00<00:00, 53564.22it/s]
